 我们分析的目标有
- 平均宽度是多少？（一般行动力是多少）
    如果成功分析，下一步可以是：
    - 逆向ab剪枝，在1h的训练下，能够获得多少棋局的minimax value？ 能够得到多少层的信息？保存到磁盘上可不可行？
    - 正向ab剪枝，预测在numba加速的前提下，能够分析多少层？
- 出现对称剪枝的情况多不多？

 - 可以把统计数据记到回合上面，然后最后分析 开局、中局、残局的信息

In [3]:
from experiment.评估函数.simulator import Simulator
from experiment.评估函数.greedy_ai import GreedyAI
import experiment.old_ai.AI as ai
from numba import njit

import src.project1.submit.AI as new_ai

import numpy as np
import random

# chessboard_size = 4  # 4赛罗游戏
chessboard_size = 8  # 正常黑白棋游戏
time_out = 5


def random_baseline(a, p):
    return random.random()  # 0-1随机权重

# breadth = [0 for i in range(65)] # 4到64回合可以用
breadth = np.zeros(65) # 4到64回合可以用
breadth_times = np.zeros(65)
def clear_accounts():
    global breadth, breadth_times
    breadth = np.zeros(65) # 4到64回合可以用
    breadth_times = np.zeros(65)
clear_accounts()
def account_breadth(rounds, color, chessboard, agents):
    b = len(new_ai.actions(chessboard, color))
    breadth[rounds] += b
    breadth_times[rounds]+=1

In [7]:
def account_tenth():
    accountants = [account_breadth]
    for i in range(10):
        for i_color in [ai.COLOR_BLACK, ai.COLOR_WHITE]:
            agents = {i_color: GreedyAI(chessboard_size, i_color, random_baseline),
                      -i_color: GreedyAI(chessboard_size, -i_color, random_baseline)}
            simulator = Simulator(chessboard_size, time_out, agents)
            winner = simulator.quick_run(accountants, True)

In [8]:
account_tenth()

过
过
过
过
过
过
过
过
过
过
过
过


In [10]:
print(breadth)
print(breadth_times)    # 第六十四回合永远terminal，所以nan

[  0.   0.   0.   0. 160. 120. 185. 202. 230. 243. 285. 298. 322. 325.
 359. 357. 364. 399. 389. 414. 440. 423. 461. 451. 456. 486. 461. 474.
 491. 479. 497. 469. 516. 490. 519. 495. 510. 469. 487. 463. 483. 445.
 464. 434. 430. 405. 420. 362. 380. 340. 339. 306. 300. 267. 278. 222.
 212. 183. 166. 139. 113.  89.  66.  40.   0.]
[ 0.  0.  0.  0. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40.
 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40.
 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40.
 40. 40. 40. 41. 40. 40. 40. 42. 46. 57.  0.]


In [19]:
avg_breadths = np.nan_to_num(breadth/breadth_times)
print(avg_breadths)

[ 0.          0.          0.          0.          4.          3.
  4.625       5.05        5.75        6.075       7.125       7.45
  8.05        8.125       8.975       8.925       9.1         9.975
  9.725      10.35       11.         10.575      11.525      11.275
 11.4        12.15       11.525      11.85       12.275      11.975
 12.425      11.725      12.9        12.25       12.975      12.375
 12.75       11.725      12.175      11.575      12.075      11.125
 11.6        10.85       10.75       10.125      10.5         9.05
  9.5         8.5         8.475       7.65        7.5         6.675
  6.95        5.55        5.3         4.46341463  4.15        3.475
  2.825       2.11904762  1.43478261  0.70175439  0.        ]


C:\Users\YeCanming\AppData\Local\Temp\ipykernel_14172\2469600728.py:1: RuntimeWarning: invalid value encountered in true_divide
  avg_breadths = np.nan_to_num(breadth/breadth_times)


In [20]:
avg_breadths.mean()

8.093369219197761